# Fysisch Geografische Regio kaart

In [1]:
import geopandas as gpd

In [2]:
gdf = gpd.read_file('../data/bronbestanden/FGR.json')

In [3]:
gdf.shape

(958, 5)

In [4]:
gdf.head()

,id,objectid,afk,fgr,geometry
0,fysischgeografischeregios.f33e0150-9af7-4a95-8...,79,du,Duinen,"POLYGON ((105147.000 526863.188, 105092.000 52..."
1,fysischgeografischeregios.c963d959-d295-492b-a...,80,du,Duinen,"POLYGON ((111314.000 552409.000, 111289.008 55..."
2,fysischgeografischeregios.cb9746da-1b31-40f7-b...,96,du,Duinen,"POLYGON ((119244.333 577690.288, 119230.924 57..."
3,fysischgeografischeregios.e3b8d3bd-e9bc-456d-a...,101,du,Duinen,"POLYGON ((133749.490 591283.817, 133726.437 59..."
4,fysischgeografischeregios.2b3f47fb-2acb-4b6b-9...,108,du,Duinen,"POLYGON ((164145.044 606590.552, 164118.586 60..."


### Check validity

In [5]:
all(gdf['geometry'].is_valid)

True

In [6]:
any(gdf['geometry'].isna())

False

In [7]:
any(gdf['geometry'].is_empty)

False

### Check self-overlap

Er lijkt zelf overlap te zijn tussen shapes in de FGR.
Deze overlap lijkt een afrondfout, de oppervlaktes zijn minimaal.

In [69]:
res_intersection = gdf.overlay(gdf, how='intersection')

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/geopandas/geodataframe.py:2323: UserWarning: `keep_geom_type=True` in overlay resulted in 3160 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [75]:
res_intersection = res_intersection[res_intersection.id_1 != res_intersection.id_2]

In [76]:
res_intersection.shape

(8, 9)

In [78]:
res_intersection.geometry.area

24     0.000706
25     0.000577
26     0.000568
27     0.003466
771    0.000706
808    0.000577
812    0.000568
818    0.003466
dtype: float64

In [79]:
res_intersection.to_file('../data/bronbestanden/fgr_overlap.shp')

### Oppervlak per FGR type

Ik vind alle FGR gebieden terug die genoemd worden in de definitietabel.

In [80]:
gdf['area'] = gdf.geometry.area

In [81]:
gdf.groupby('fgr').size()

fgr
Afgesloten Zeearmen      9
Duinen                 122
Getijdengebied           3
Heuvelland              11
Hogere Zandgronden     307
Laagveengebied         117
Niet indeelbaar        239
Noordzee                 1
Rivierengebied          25
Zeekleigebied          124
dtype: int64

In [82]:
gdf.groupby('fgr')['area'].apply(lambda x: round(sum(x)/100000)).rename('aantal hectares per FGR categorie')

fgr
Afgesloten Zeearmen     25567
Duinen                   9097
Getijdengebied          32937
Heuvelland               5095
Hogere Zandgronden     163478
Laagveengebied          25594
Niet indeelbaar         11579
Noordzee               594770
Rivierengebied          36187
Zeekleigebied           95981
Name: aantal hectares per FGR categorie, dtype: int64